In [32]:
import tensorflow as tf
import numpy as np
import os
import time
import matplotlib.pyplot as plt
import re

In [4]:
text = open(r'C:\Users\ALLARASSEMJJ20\big_rome.txt', 'rb').read().decode(encoding='utf-8')

In [5]:
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

177 unique characters


In [9]:
vocab[:10]
example_texts = ['I go to school', 'xyz']
chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')

In [10]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

ids_from_chars(chars)

<tf.RaggedTensor [[36, 3, 64, 72, 3, 77, 72, 3, 76, 60, 65, 72, 72, 69], [81, 82, 83]]>

In [11]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)
chars_from_ids(139)

<tf.Tensor: shape=(), dtype=string, numpy=b'\xce\xbb'>

In [8]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [9]:
chars_from_ids(12)
ids = ids_from_chars(chars)
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'I', b' ', b'g', b'o', b' ', b't', b'o', b' ', b's', b'c', b'h', b'o',
  b'o', b'l']                                                            ,
 [b'x', b'y', b'z']]>

In [12]:
all_chars = tf.strings.unicode_split(text, input_encoding='UTF-8')
all_ids = ids_from_chars(all_chars)
all_ids

<tf.Tensor: shape=(10608444,), dtype=int64, numpy=array([177,   3,   3, ...,  66,  60,  12], dtype=int64)>

In [13]:
seq_length = 200
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

In [12]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

﻿
 
 
 
 
 
 
T
h
e


In [12]:
for seq in sequences.take(2):
    print(chars_from_ids(seq))

tf.Tensor(
[b'\xef\xbb\xbf' b' ' b' ' b' ' b' ' b' ' b' ' b'T' b'h' b'e' b' ' b'g'
 b'r' b'e' b'a' b't' b' ' b'w' b'o' b'r' b'k' b' ' b'o' b'f' b' ' b'G'
 b'i' b'b' b'b' b'o' b'n' b' ' b'i' b's' b' ' b'i' b'n' b'd' b'i' b's'
 b'p' b'e' b'n' b's' b'a' b'b' b'l' b'e' b' ' b't' b'o' b' ' b't' b'h'
 b'e' b' ' b's' b't' b'u' b'd' b'e' b'n' b't' b' ' b'o' b'f' b'\r' b'\n'
 b' ' b' ' b' ' b' ' b' ' b' ' b'h' b'i' b's' b't' b'o' b'r' b'y' b'.'
 b' ' b'T' b'h' b'e' b' ' b'l' b'i' b't' b'e' b'r' b'a' b't' b'u' b'r'
 b'e' b' ' b'o' b'f' b' ' b'E' b'u' b'r' b'o' b'p' b'e' b' ' b'o' b'f'
 b'f' b'e' b'r' b's' b' ' b'n' b'o' b' ' b's' b'u' b'b' b's' b't' b'i'
 b't' b'u' b't' b'e' b' ' b'f' b'o' b'r' b' ' b'\xe2\x80\x9c' b'T' b'h'
 b'e' b'\r' b'\n' b' ' b' ' b' ' b' ' b' ' b' ' b'D' b'e' b'c' b'l' b'i'
 b'n' b'e' b' ' b'a' b'n' b'd' b' ' b'F' b'a' b'l' b'l' b' ' b'o' b'f'
 b' ' b't' b'h' b'e' b' ' b'R' b'o' b'm' b'a' b'n' b' ' b'E' b'm' b'p'
 b'i' b'r' b'e' b'.' b'\xe2\x80\x9d' b' ' b'I' b't' b' ' b'h

In [14]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [14]:
for el in dataset.take(5):
    print(el)

(<tf.Tensor: shape=(200,), dtype=int64, numpy=
array([177,   3,   3,   3,   3,   3,   3,  47,  65,  62,   3,  64,  75,
        62,  58,  77,   3,  80,  72,  75,  68,   3,  72,  63,   3,  34,
        66,  59,  59,  72,  71,   3,  66,  76,   3,  66,  71,  61,  66,
        76,  73,  62,  71,  76,  58,  59,  69,  62,   3,  77,  72,   3,
        77,  65,  62,   3,  76,  77,  78,  61,  62,  71,  77,   3,  72,
        63,   2,   1,   3,   3,   3,   3,   3,   3,  65,  66,  76,  77,
        72,  75,  82,  12,   3,  47,  65,  62,   3,  69,  66,  77,  62,
        75,  58,  77,  78,  75,  62,   3,  72,  63,   3,  32,  78,  75,
        72,  73,  62,   3,  72,  63,  63,  62,  75,  76,   3,  71,  72,
         3,  76,  78,  59,  76,  77,  66,  77,  78,  77,  62,   3,  63,
        72,  75,   3, 172,  47,  65,  62,   2,   1,   3,   3,   3,   3,
         3,   3,  31,  62,  60,  69,  66,  71,  62,   3,  58,  71,  61,
         3,  33,  58,  69,  69,   3,  72,  63,   3,  77,  65,  62,   3,
        45,  72, 

In [15]:
# Batch size
BATCH_SIZE = 300

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(300, 200), dtype=tf.int64, name=None), TensorSpec(shape=(300, 200), dtype=tf.int64, name=None))>

In [16]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 512

In [17]:
class MyModel(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, rnn_units):
        super().__init__(self)
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(rnn_units,
                                       return_sequences=True,
                                       return_state=True)
        self.dense = tf.keras.layers.Dense(vocab_size)
        
    def call(self, inputs, states=None, return_state=False, training=False):
        x = inputs
        x = self.embedding(x, training=training)
        if states is None:
            states = self.gru.get_initial_state(x)
        x, states = self.gru(x, initial_state=states, training=training)
        x = self.dense(x, training=training)

        if return_state:
            return x, states
        else:
            return x

In [18]:
model = MyModel(
    vocab_size = vocab_size, 
    embedding_dim = embedding_dim, 
    rnn_units = rnn_units
)

In [ ]:
models.load_

In [19]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(300, 200, 178) # (batch_size, sequence_length, vocab_size)


In [20]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  45568     
                                                                 
 gru (GRU)                   multiple                  1182720   
                                                                 
 dense (Dense)               multiple                  91314     
                                                                 
Total params: 1,319,602
Trainable params: 1,319,602
Non-trainable params: 0
_________________________________________________________________


In [21]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [22]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (300, 200, 178)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(5.1831093, shape=(), dtype=float32)


In [23]:
model.compile(optimizer='adam', loss=loss)

In [24]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints_rome'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [25]:
model

In [27]:
EPOCHS = 100

In [28]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/100
 39/175 [=====>........................] - ETA: 14:52 - loss: 3.6588

KeyboardInterrupt: 

In [48]:
models = MyModel(
    vocab_size = vocab_size, 
    embedding_dim = embedding_dim, 
    rnn_units = rnn_units
)
models50 = MyModel(
    vocab_size = vocab_size, 
    embedding_dim = embedding_dim, 
    rnn_units = rnn_units
)
models10 = MyModel(
    vocab_size = vocab_size, 
    embedding_dim = embedding_dim, 
    rnn_units = rnn_units
)

In [61]:
latest_checkpoint = tf.train.latest_checkpoint('./training_checkpoints_rome')  # Get the path of the latest checkpoint
models.load_weights(latest_checkpoint)

In [50]:
models50.load_weights('./training_checkpoints_rome/ckpt_50')
models10.load_weights('./training_checkpoints_rome/ckpt_50')

In [37]:
models

In [19]:
def generate_one(inputs, states, model):
    # turn list of string into a tensorflow ragged vector
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    
    # turn that tensorflow ragged object into a list of ids
    input_ids = ids_from_chars(input_chars).to_tensor()
    
    # get the digits and the next state
    predicted_logits, states = model(inputs=input_ids, states=states,
                                          return_state=True)
    
    # pick only the last digit (batch, char, next character)
    predicted_logits = predicted_logits[:, -1, :]
    
    # sample from the categorical to avoid loops 
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)
    
    # get list of characters from list of ids
    predicted_chars = chars_from_ids(predicted_ids)
    return predicted_chars, states

def generate_many(seed, model, n):
    next_char = tf.constant([seed])
    result = [next_char]
    states = None
    for n in range(n):
        next_char, states = generate_one(next_char, states, model)
        result.append(next_char)
    result = tf.strings.join(result)
    return result[0].numpy().decode('utf-8')

#print(generate_many('Rome is the ', models, 500))

In [20]:
model50 = MyModel(
    vocab_size = vocab_size, 
    embedding_dim = embedding_dim, 
    rnn_units = rnn_units
)

In [ ]:
model50 = load_weights()

In [69]:
for i in range(1, 99, 5):
    model = MyModel(
    vocab_size = vocab_size, 
    embedding_dim = embedding_dim, 
    rnn_units = rnn_units
    )
    model.load_weights(f'./training_checkpoints_rome/ckpt_{i}')
    print(f'EPOCH IS {i} :\n',generate_many('Rome is the ', model, 500))

EPOCH IS 1 :
 Rome is the seecithar obithe Meind wides  casthid dioud tir s. Ariculist the
      Grmepar ans and tuaelest bit wabaigea daina, pereders anatisn s thawoperatulgel, of the s bompoirce Bly, he almixthe as
      ifad the shretole: and ied  ithe Rarce ths hlatithale ppeas ceem,
      med so mianf Hhed the thigrtas menc the shery som  re on gors torrein the  actes (ntuslcos,4”) os progat, e ded aed at prirnnan; th
      aen) _,or tire
      ojs
      rutho
      pulegicenerdrphinns expopesugns wor, vhe ja
EPOCH IS 6 :
 Rome is the bornful never
      fir miles could no complained the quast discific the appenage
      specious populous was unknown tyrant. His imurate is
      armied, and Nice is still native provized the satisfician legiths
      were insplained oth obtained with instoned to the confiness
      of consequence, (tom. ii. p. 317-500, p. 74: 30.—M.]

      61 (return) [ Judicausles Challescanden, (l. xii. p. 567.]

      22 (return) [ I paract in this oragen prote

In [70]:
print(generate_many('Rome is ', models, 500))

Rome is expressed with legal tongue
      would have been so cheached without expectation or intervals
      understard the fluctuation of such months of the Latin blood
      and people; and his rival pride were satisfied with the loss of
      fanaticism dissented a genuine rife of nature and energy at
      the madness of the crusades; and the councils was successively
      intimately conferred on themselves on their hand to his foel a
      rude spare to reverend the blood of the Latins; he 


In [71]:
print(generate_many('The rise of ', models, 500))

The rise of Khile, some
      knowledge of Publius is a larger proselytes of the Moguls, or
      then embarked with the allowance of Arabia; nor were the lawful
      followers of idolatry; but the license of deputing with leading
      a wild extremity, the wealth and ruin into a republic liwed from
      endolf. He seized the name and invitation of Sixtue tribes, with
      _vulgarians of peace, turns the presence of his brother would
      do the conswerness of Magnesia. 8 Note: Simere were not i


In [72]:
print(generate_many('Rome, Syria, Lybia', models, 500))

Rome, Syria, Lybias
      who had used in the fertile Imperial style of the city, which
      endangered their absolvation with their wants and adopting Oriental
      conquests. Hell is one of them as considerable by the clamors of
      the wind. 21 In this supposita quitatus Malthernit. P. vi. 216.
      The ancient creed of Homer would have inculcated, a defeat of
      Jerusalem, as he saught in the islands and castles of the church,
      repeated above vicinus had a solemn proof, his ardor was infor


In [73]:
print(generate_many('104 (', models, 500))

104 (return) [ It was that the count of Sapor, afforded by silk
      of exigerate Othman, at the end of Maclesus praised extorted by
      the barren-life of a Scythian guard, (Basilleux. l. v. c. 18—17,) who
      swellous thems, _Whathorimos_ totey passagres the Church,” says
      Mosheim (A.D. 1054,) Klaprother, the propper that supposes for
      Gregory of Tours, I he meditate the curiosity of the
      posts.” The ministers of Pharamel, the Ottoman grant and pressing
      soldiers, th


In [75]:
print(generate_many('104 [', models, 700))

104 [ Ammianus soughtly gathered a deep and authentic.]

      91 (return) [ Horace tide; how in such religion passed the
      idleness of their master and the loss of their family, was
      established by the fugitive; and the king of the North and horses
      are still alive in a pride or the indigent title of Baldwin of
      Kermanhal art. The text of the Alecand, the Vatican, who gives
      the resource of the seabous Virto, it usurpated by the intent
      of rhetoric and of the Tigriar, patience do not be placed, as
      we cannot built or inglorious, my landing between Wallk or
      persecution to his eyes are thrown open to cue the habits of
      sharp character, they w


In [76]:
print(generate_many('Alex', models, 500))

Alexiast
      Lampasmer, one of the most ancient Barbarians_, at 25 and the
      saint in Italy, that were taught against the common hero of
      the caliphs, he conjured the fears and garacles of Arabs, had
      torn one hundred and twenty soldiers, the Colognaish of the
      Romans, in their fortune, or the reward, he might prove the
      possession of the influence of volunteers; the attacks of the
      reigning emperor, after the duty of embroidery was rather executed
      by the


In [77]:
print(generate_many('10 (', models, 500))

10 (return) [ Sometimes stipulate the titles of government; and
      their zealous words, Raymond Of Mahomet declared him to surney
      and kill. Atathete text, undester the system of written tenerat, a
      wealthy army of critical advice, he enjoyed this orthodox
      Christian femalish demolitions, had repeatedly varied against
      five thousaphy. His military light of their faithful and modest
      penalties were overpowered, and were unwilling or corrupted: the
      remains of h


In [80]:
print(generate_many('East and  ',models, 500))

East and  Arcadius was paid from the city; the approach
      of monks, who, in the supply of this important philosophy,
      accepting the prodigious enterprise of the Chronicle of PaganismÕs
      edition, (Novell. Mongout. p. 70.)]

      201 (return) [

     III. That an epistle see the emperor of Rome and Africa,
      has persuaded 21 or septensits was offered to a free pontiff;
      resolved to applaud the dignity of ordination, ignorant of his
      brethren who had prostituted the nat


In [81]:
print(generate_many('(retu', models, 500))

(return) [ Jesii Vintteres sons as Samarcand and Isacis to
      the Atlance, we pass forty, enthusiasts, the dynasty of windows
      and chastity on the a line of Picenus by the promised to the Italian
      amount of the times, that aeray or Aramus, a pestilential
      body was invested by the Dracelets, the levity of his slain
      had obtained the smooth country of the Teutonic idiom: the affection
      of Tartars and Protes, who, in a moment of desire without
      danger from east ho


In [82]:
print(generate_many('(', models, 500))

(return) [ On the result of his undaunted march from Paris,
      and Evenized, whith the reking of the villages could not be
      delivered into the field. Ans bellowing are the measure of his
      largest object, he speaks of the Imperial day, the
      ample proselytes of Constantinople subjects his denic est in
      prayer and the great duke.]

      48 (return) [ Anna Comnena, (Paul Warsel,) and Theophamet Per Wilf,
      polisâ, see Diariæ Maffeires. Græci, Innocent pro Modalum v


In [85]:
model99 = models

In [86]:
model99.compile(optimizer='adam', loss=loss)

In [87]:
history = model99.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/100
175/175 [==============================] - 6355s 36s/step - loss: 0.8359
Epoch 2/100
175/175 [==============================] - 4846s 28s/step - loss: 0.8358
Epoch 3/100
175/175 [==============================] - 1377s 8s/step - loss: 0.8353
Epoch 4/100
175/175 [==============================] - 1341s 8s/step - loss: 0.8343
Epoch 5/100
175/175 [==============================] - 1340s 8s/step - loss: 0.8343
Epoch 6/100
175/175 [==============================] - 1384s 8s/step - loss: 0.8339
Epoch 7/100
175/175 [==============================] - 1825s 10s/step - loss: 0.8337
Epoch 8/100
175/175 [==============================] - 5555s 32s/step - loss: 0.8333
Epoch 9/100
175/175 [==============================] - 1344s 8s/step - loss: 0.8326
Epoch 10/100
175/175 [==============================] - 1359s 8s/step - loss: 0.8324
Epoch 11/100
175/175 [==============================] - 16140s 93s/step - loss: 0.8319
Epoch 12/100
175/175 [==============================] - 1152s 7s/ste

KeyboardInterrupt: 

In [88]:
print(generate_many('(retu', model99, 500))

(return) [ Lactantius de Mogium, dissertat. Pariss, p. 112.
      Augustus ma Germany humagen, Volusius and Zobeirs to the year
      of the Bletius, and three temples, of rebellion of age, by
      the particulars, and after eighty stadia, or seventeen with
      points of which; ®tius confirms the expedition of the times;
      and elpirh, those wan their approbation, was in favor of Constantine,
      and restored the doctrines of manhagres; and on the superior
      ruin, which might envy,


In [90]:
print(generate_many(' ', models, 500))

 which was
      enabled to purchase, indeed, were shut, they themselves, after a
      resistance against the freedom and sentiments of a rigorous
      error. Five leagues, as they were respected as the only theory
      of disengagan. 91 The zeal of the same perpetual Tyai, And
      Ammianus, with their ancient ancite to murder or it with
      food and acknowledge than on their young Carpastine Plato, while
      his grief was less credit.]

      32 (return) [ When Hansides lay rou


In [21]:
BATCH_SIZE = 100
EPOCHS=100

In [22]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints_rome_part_3'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [23]:
model_219 = MyModel(
    vocab_size = vocab_size, 
    embedding_dim = embedding_dim, 
    rnn_units = rnn_units
)

In [59]:
model_2 = MyModel(
    vocab_size = vocab_size, 
    embedding_dim = embedding_dim, 
    rnn_units = rnn_units
)

In [60]:
model_2.load_weights('./training_checkpoints_rome/ckpt_2')

In [25]:
model_219.load_weights('./training_checkpoints_rome_part_3/ckpt_56')

In [45]:
dictionnary = set(text.split(' '))

def hallucination_rate(seq):
    r = 0
    punctuation_pattern = r"[^\w\s]"
    seq = re.sub(punctuation_pattern, "", seq)
    seq = seq.split(' ')
    seq = [w.strip() for w in seq]
    for s in seq:
        if s not in dictionnary:
            r = r +1
            print("bad: ", s)
    return r/len(seq)

'contrary' in dictionnary

True

In [65]:
s = generate_many('Chapter I: Rome', model_2, 500)
print(s)
print(hallucination_rate(s))

Chapter I: Rome
      deceas insolitulen, In the deadows of the hecresers of Ch. 481,) Cersim, co quysmest. In Ullix, ippiatius. p.
      Sitariny of their
      Slatial Appronative sancersat of the obelus in be _excatropy mo stornat as
      Miram. His Suldide of marmand sucplay of the
      Jonstale ofretidnagiow kiphings, and vasurstry. The
      mide in benaks Delusicid Constouthone and itatonn tho
      proccove to Thenks cokzemster frove fork hid ispely Sles
      geroment of Byedk of a caufess w
bad:  deceas
bad:  insolitulen
bad:  deadows
bad:  hecresers
bad:  Cersim
bad:  quysmest
bad:  Ullix
bad:  ippiatius
bad:  Sitariny
bad:  Slatial
bad:  Appronative
bad:  sancersat
bad:  obelus
bad:  _excatropy
bad:  stornat
bad:  Miram
bad:  Suldide
bad:  marmand
bad:  sucplay
bad:  Jonstale
bad:  ofretidnagiow
bad:  kiphings
bad:  vasurstry
bad:  mide
bad:  benaks
bad:  Delusicid
bad:  Constouthone
bad:  itatonn
bad:  tho
bad:  proccove
bad:  Thenks
bad:  cokzemster
bad:  frove
bad:  fo

In [ ]:
s = generate_many('J', model_219, 1000)
print(s)
print(hallucination_rate(s))

In [70]:
model_219.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=.0001), loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True))

In [72]:
history = model_219.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/100
175/175 [==============================] - 1356s 8s/step - loss: 0.7716
Epoch 2/100
175/175 [==============================] - 28465s 164s/step - loss: 0.7726
Epoch 3/100
175/175 [==============================] - 1386s 8s/step - loss: 0.7726
Epoch 4/100
175/175 [==============================] - 1361s 8s/step - loss: 0.7725
Epoch 5/100
 39/175 [=====>........................] - ETA: 17:12 - loss: 0.7664

KeyboardInterrupt: 